In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
mpl.rc('font', size=12)

from numba import njit

# Voter model with mutation, coalescence approach

In [43]:
@njit
def init_grid(L):
    '''Initialises an L by L grid with numbers drawn randomly from (0, 1)'''
    return np.random.rand(L, L)

In [44]:
@njit
def get_4_neighbors(i, j, L):
    '''Finds upper, lower, left and right neighbor of a site in a K by K grid'''
    neighbors = []
    # Up, down, right, left
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  
    for di, dj in directions:
        # Apply periodic boundary conditions
        neighbor_i = int((i + di) % L)
        neighbor_j = int((j + dj) % L)
        neighbors.append((neighbor_i, neighbor_j))
    return neighbors

In [45]:
def sa_curve(grid):
    height, width = grid.shape
    
    n_centers = 10
    centers_x = np.random.choice(np.arange(0, width), n_centers) + width
    centers_y = np.random.choice(np.arange(0, height), n_centers) + height

    areas = []
    species = []
    
    torus_grid = np.vstack((grid, grid, grid))
    torus_grid = np.hstack((torus_grid, torus_grid, torus_grid))
        
    for i, (x, y) in enumerate(zip(centers_x, centers_y)):
        cur_species = []
        for j in range(width//2):
            cur_species.append(len(np.unique(torus_grid[x-j:x+j+1, y-j:y+j+1])))
            if i == 0:
                areas.append((j+1)**2)
        species.append(cur_species)
    
    return areas, species

In [46]:
import random
import itertools
from tqdm import tqdm
import time

# @njit
def voter_model_fast(L, alpha, MC_steps, rand_walkers):
    '''Run experiment with the voter model
    Inputs:
    grid_0 (numpy array): Initial grid
    alpha (float): Value of alpha parameter
    MC_steps (int): number of monte carlo steps
    
    Returns:
    cur_grid (numpy array): Grid after n_iter iterations
    num_species (list): Contains amount of different species at each tenth iteration
    '''
    # Create list to store number of unique species
    lineages = [set() for _ in range(len(rand_walkers))]
    for i, element in enumerate(rand_walkers):
        lineages[i].add(element)

    rand_walker_pos = list(rand_walkers.copy())
    species = []
    
    for i in range(MC_steps):
        # Draw full list of random numbers to save time
        rand_nums = np.random.uniform(size=L**2)
        for j in tqdm(range(L**2)):
            if len(rand_walker_pos) == 0:                
                return lineages, species
            
            # Select active walker
            cur_walker_loc = np.random.choice(len(rand_walker_pos))
            cur_walker = rand_walker_pos[cur_walker_loc]
            # Find current walker's place in lineages
            cur_walker_idx = L*cur_walker[0] + cur_walker[1]
            
            # Select parent
            potential_parents = get_4_neighbors(cur_walker[0], cur_walker[1], L)
            parent_walker_loc = np.random.choice(len(potential_parents))
            parent_walker = potential_parents[parent_walker_loc]
            # Find parent's place in lineages
            parent_walker_idx = L*parent_walker[0] + parent_walker[1]
            
            # Speciate with probability alpha
            if rand_nums[j] < alpha:
                species.append(lineages[cur_walker_idx])
                # Insert pointer to correct species where the walker is in
                for site in lineages[cur_walker_idx]:
                    site_idx = L*site[0] + site[1]
                    lineages[site_idx] = len(species) - 1                
            else:
                # Check if parent hasn't speciated yet
                if type(lineages[parent_walker_idx]) == set:
                    # Merge lineages if walkers are from other walks
                    if parent_walker not in lineages[cur_walker_idx]:
                        # Unify lineages of parent and child
                        for site in lineages[cur_walker_idx]:
                            site_idx = L*site[0] + site[1]
                            lineages[site_idx] = lineages[site_idx].union(lineages[parent_walker_idx])
                            
                        for site in lineages[parent_walker_idx]:
                            site_idx = L*site[0] + site[1]
                            if type(lineages[site_idx]) == set:
                                lineages[site_idx] = lineages[site_idx].union(lineages[cur_walker_idx])
                                
                    # Handle case when walker has moved back into its own lineage
                    else:
                        # Change position of active walker
                        if parent_walker not in rand_walker_pos:
                            rand_walker_pos.append(parent_walker)
                            
                # Let speciation occur
                elif type(lineages[parent_walker_idx]) == int:
                    species[lineages[parent_walker_idx]] = species[lineages[parent_walker_idx]].union(lineages[cur_walker_idx])
                    for site in lineages[cur_walker_idx]:
                        site_idx = L*site[0] + site[1]
                        lineages[site_idx] = lineages[parent_walker_idx]
            
            # Remove current walker from list of active walkers
            rand_walker_pos.remove(cur_walker)

    return lineages, species

In [ ]:
# Define parameters as in paper
alpha = 10e-4
L = 100

MC_steps = int(5000)
rand_walkers = list(itertools.product(range(L), range(L)))

lineages, species = voter_model_fast(L, alpha, MC_steps, rand_walkers)

 36%|████████████▊                       | 3560/10000 [00:00<00:00, 9890.27it/s]

In [ ]:
species_ids = np.random.uniform(size=len(species))

grid = np.zeros((L, L))
for i, specie in enumerate(species):
    for coord in specie:
        grid[coord] = species_ids[i]

In [ ]:
plt.imshow(grid)
plt.show()

In [ ]:
areas, species = sa_curve(grid)
spec_std_dev = np.std(species, axis=0)
spec_mean = np.mean(species, axis=0)

poly_coeffs = np.polyfit(np.log(areas), np.log(spec_mean), 1)
print(poly_coeffs)

plt.loglog(areas, spec_mean, label='Mean of 10 centers')
plt.loglog([areas[0], areas[-1]], 
           np.exp(poly_coeffs[1]) * np.array([areas[0], areas[-1]])**poly_coeffs[0], 
           color='grey', 
           linestyle='dashed',
           label=f'Lin. regress, power={round(poly_coeffs[0], 2)}')
plt.fill_between(areas, spec_mean-spec_std_dev, spec_mean+spec_std_dev, alpha=0.2, label='Std. dev')
plt.ylabel('Log Number of Species')
plt.xlabel('Log Area')
plt.title(f'Species area curve for L={L}')
plt.legend()
plt.show()